## Make maps of temperature trends over the alternate climate divisions
### uses data from NCEI nClimGrid, averaged over the divisions, and with trends calculated with the Theil-Sen trend estimator

In [ ]:
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.dates as mdates 
import matplotlib.colors as mcolors
import numpy as np
import os

from metpy.plots import USCOUNTIES
from matplotlib.offsetbox import AnchoredText

from palettable.colorbrewer.diverging import BrBG_11


### read in cluster definitions, make sure they look right

In [ ]:
mask_in = xr.open_dataset("data/clusters_tavg_prcp_sum_post1950_11_pub.nc")['clusters_tavg_prcp_sum']

mask_in.plot.contourf(levels=range(0,12))

### dictionary of the divisions

In [ ]:
divs_dict = {1:"Pikes Peak",2:"Southeast",3:"N Front Range",4:"Northeast",5:"South Park",
             6:"San Luis Valley & SdC",7:"Southwest",8:"Mesas & Valleys",9:"Northwest",
             10:"Northern Mountains",11:"Central Mountains",
             "state":"statewide"}

### read in the trend data, and plot it on a map

In [ ]:
### make list of calculations to loop over
calcs = ["Tchange_sen_climdiv","Tchange_sen_climdiv_since1980",
         "Tchange_seas_sen_climdiv","Tchange_seas_sen_climdiv_since1980",
         "Tmaxchange_sen_climdiv","Tmaxchange_sen_climdiv_since1980",
         "Tmaxchange_seas_sen_climdiv","Tmaxchange_seas_sen_climdiv_since1980",
         "Tminchange_sen_climdiv","Tminchange_sen_climdiv_since1980",
         "Tminchange_seas_sen_climdiv","Tminchange_seas_sen_climdiv_since1980"]

time_period = {"Tchange_sen_climdiv":"1895-2022",
               "Tchange_sen_climdiv_since1980":"1980-2022",
              "Tchange_seas_sen_climdiv":"1895-2022",
              "Tchange_seas_sen_climdiv_since1980":"1980-2022",
               "Tmaxchange_sen_climdiv":"1895-2022",
               "Tmaxchange_sen_climdiv_since1980":"1980-2022",
              "Tmaxchange_seas_sen_climdiv":"1895-2022",
              "Tmaxchange_seas_sen_climdiv_since1980":"1980-2022",
               "Tminchange_sen_climdiv":"1895-2022",
               "Tminchange_sen_climdiv_since1980":"1980-2022",
              "Tminchange_seas_sen_climdiv":"1895-2022",
              "Tminchange_seas_sen_climdiv_since1980":"1980-2022"}

titles = {"Tchange_sen_climdiv":"average temperature",
               "Tchange_sen_climdiv_since1980":"average temperature",
              "Tchange_seas_sen_climdiv":"average temperature",
              "Tchange_seas_sen_climdiv_since1980":"average temperature",
              "Tmaxchange_sen_climdiv":"average high temperature",
               "Tmaxchange_sen_climdiv_since1980":"average high temperature",
              "Tmaxchange_seas_sen_climdiv":"average high temperature",
              "Tmaxchange_seas_sen_climdiv_since1980":"average high temperature",
              "Tminchange_sen_climdiv":"average low temperature",
               "Tminchange_sen_climdiv_since1980":"average low temperature",
              "Tminchange_seas_sen_climdiv":"average low temperature",
              "Tminchange_seas_sen_climdiv_since1980":"average low temperature"}  

month_titles = {"Jan":"January","Feb":"February","Mar":"March","Apr":"April",
                "May":"May","Jun":"June","Jul":"July","Aug":"August","Sep":"September",
                "Oct":"October","Nov":"November","Dec":"December","annual":"annual",
                "DJF":"winter","MAM":"spring","JJA":"summer","SON":"autumn"}

for calc in calcs:
    
    ### read csv with trends (calculated in the 'heatmaps' notebook)
    trends = pd.read_csv("data/trend_"+calc+".csv")
    
    for month in trends.columns.values[1:]:

        ### set the values of the map to be the trend for each division
        trend_mapped = mask_in.copy()

        ### loop over the divisions
        for div in range(1,12):
            trend_mapped = xr.where(trend_mapped==div,trends[trends.division==divs_dict[div]][month].values[0],trend_mapped)

        crs = ccrs.LambertConformal(central_longitude=-105.0, central_latitude=40.)
        #crs = ccrs.LambertConformal(central_longitude=-100.0, central_latitude=42.0)

        fig = plt.figure(figsize=(12,8))
        ax = fig.add_subplot(1,1,1,projection=crs)

        ## Colorado versions
        lonmin=-109.5
        lonmax=-101.5
        latmin=36.4
        latmax=41.5
        #lonmin=-109.046667
        #lonmax=-102.046667
        #latmin=37.0
        #latmax=41.0

        ax.set_extent([lonmin,lonmax,latmin,latmax])
        #ax.add_feature(cfeature.LAND)
        ax.add_feature(USCOUNTIES.with_scale('5m'), edgecolor="gray", linewidth=0.4)
        ax.add_feature(cfeature.STATES)
        ax.add_feature(cfeature.BORDERS)

        ## trend
        cf1 = ax.contourf(trend_mapped.lon, trend_mapped.lat, trend_mapped, 
                          #np.arange(0,5.25,0.25), extend='both',
                          #cmap="Reds",
                          np.arange(-5,5.25,0.25), extend='both',
                          cmap="RdBu_r",
                          transform=ccrs.PlateCarree())

        ## also lightly outline the divisions -- kind of hacky masking to make the contours all the same thickness
        for div in range(1,12):
            mask_temp = xr.where(mask_in==div,1,0)
            ax.contour(mask_temp.lon, mask_temp.lat, mask_temp,
                      #levels=[0],
                       colors='gray',
                       linewidths=0.15,
                      transform=ccrs.PlateCarree())

        ### and add text to each division
        for div in range(1,12):

            if (div==2) or (div==4):  ## define horizontal alignment
                hor_align = 'left'
            elif (div==3):
                hor_align = "right"
            else:
                hor_align = "center"

            if (div==7): 
                vert_align = "top"
            else:
                vert_align = "center"

            cent_lon = mask_in.where(mask_in==div,drop=True).lon.mean()
            cent_lat = mask_in.where(mask_in==div,drop=True).lat.mean()
            ax.text(cent_lon,cent_lat,
                   # str(np.round(trends[trends.division==divs_dict[div]]['annual'].values[0],1))+"%",
                    f'{trends[trends.division==divs_dict[div]][month].values[0]:+.1f}'+"°F",
                   color='maroon',fontweight='bold', fontsize=14,
                    horizontalalignment=hor_align, verticalalignment=vert_align,
                    bbox=dict(facecolor='white', alpha=0.6,boxstyle='round'),
                    transform=ccrs.PlateCarree())

        # Add statewide value
        text_state = AnchoredText("statewide: "+f'{trends[trends.division=="statewide"][month].values[0]:+.1f}'+"°F",
                            loc='upper center', prop={'size': 14,'weight':'semibold','color':'maroon'}, frameon=True)
        ax.add_artist(text_state)

        ax.set_title(titles[calc]+" change, "+month_titles[month]+", "+time_period[calc], fontsize=14, fontweight='semibold')
        cb1 = fig.colorbar(cf1, ax=ax, orientation='horizontal', aspect=30, shrink=0.65, pad=0.01)
        cb1.set_label('trend (°F)', size='large')

        text = AnchoredText("data source: NOAA/NCEI nclimgrid",
                            loc='lower left', prop={'size': 8.15}, frameon=True)
        ax.add_artist(text)

        fig.savefig(calc+"_"+month+"_map.png",
                   dpi=250,transparent=False, facecolor='white', bbox_inches='tight')

        #plt.show()

